In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import zipfile
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [6]:
zf = zipfile.ZipFile('C:/Users/Serueja/Downloads/house-prices-advanced-regression-techniques.zip')
test = pd.read_csv(zf.open('test.csv'))
train = pd.read_csv(zf.open('train.csv'))
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
submission_sample = pd.read_csv(zf.open('sample_submission.csv'))
submission_sample
X = pd.get_dummies(X)
X.fillna(value=0, inplace=True)
#X.dropna(axis=0, inplace=True)

In [7]:
Z_test = test.copy()
Z_test = pd.get_dummies(Z_test)
Z_test.fillna(value=0, inplace=True)

In [8]:
difference = set(X.columns).difference(set(Z_test.columns))

In [9]:
X.drop(list(difference), axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
model_dec_tree = tree.DecisionTreeClassifier()
parameters = {'max_depth':range(20, 80), 'criterion':['gini', 'entropy', 'log_loss']}
grid_search_cv_clf=GridSearchCV(model_dec_tree, parameters, cv=5)

In [7]:
grid_search_cv_clf.fit(X_train, y_train)

c:\Users\Serueja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(20, 80)})

In [8]:
print(grid_search_cv_clf.best_params_)
best_max_depth_dec_tree_clf = grid_search_cv_clf.best_params_
best_max_depth_dec_tree_model = grid_search_cv_clf.best_estimator_
d = {'PredictedSales':best_max_depth_dec_tree_model.predict(X_test), 'ActualSales':y_test}
y_dec_tree_26_depth=pd.DataFrame(data=d, index=np.arange(len(y_test.index)))

{'criterion': 'gini', 'max_depth': 61}


In [9]:
best_max_depth_dec_tree_model.score(X_train, y_train)
y_pred = best_max_depth_dec_tree_model.predict(X_test)
precision_score(y_pred, y_test, average='weighted', zero_division=1)

0.5205479452054794

In [10]:
super_prediction=best_max_depth_dec_tree_model.predict(Z_test)
super_prediction=pd.Series(super_prediction)
sub = pd.DataFrame({'Id':test['Id'], 'SalePrice':super_prediction})
sub.to_csv('submission.csv', index=False)